In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.context import SparkConf
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import pyspark.sql.types as T 
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, size
from operator import add
from functools import reduce
from bio_spark.io.fasta_reader import FASTAReader, FASTAQReader
import collections
import numpy as np
import sys

from pathlib import Path

from operator import add

# Sobre este Notebook

Este notebook executa uma clusterização de seuência de Aminoácidos usando a ML lib dp Spark. Clustrização é um método que pode auxiliar os pesquisadores a descobrir relações filogenéticas e/ou relações de similaridade entre sequências sem a necessidade de comparar com uma base de referência. O fluxo é composto dos seguintes passos:

1. Leutra e parsing do arquivos fasta de entrada
2. Cálculo dos Kmers a partir das sequências encontradas nos arquivos de entrada
3. Uso do método de Elbow para encontrar clusters coesos.

___

## Cluster local

Para fins de desenvolvimento, utilizamos imagens Docker para criar um cluster spark local. Esse cluster deve estar rodadndo para que o notebook funcione como esperado. Na raiz do projeto:

```shell
docker-compose up
```

In [2]:
sConf = SparkConf("spark://localhost:7077")
sc = SparkContext(conf=sConf)
spark = SparkSession(sc)

## Data Input

Tdoso os arquivos de entrada serão tratados em único Dataframe

```shell
INPUT_DIR_PATH: caminho para o diretório com os arquivs .fna (FASTA)
```

In [8]:
INPUT_DIR_PATH = Path("/home/thiago/Dados/sparkAAI-1/data/genomes/")
files_to_process = [str(f) for f in INPUT_DIR_PATH.iterdir()]
print("Files to process :", len(files_to_process))

Files to process : 11


In [9]:
# fasta_plain_df = sc.textFile(','.join(files_to_process))\/
fasta_plain_df = sc.textFile("/home/thiago/Dados/sparkAAI-1/data/genomes/Prochlorococcus_sp_W2_genomic.fna")\
            .map(lambda x: Row(row=x))\
            .zipWithIndex()\
            .toDF(["row","idx"])

print("raw file lines to process", fasta_plain_df.count())

raw file lines to process 15995


inspecionando o dataframe lido

In [10]:
fasta_plain_df.show()

+--------------------+---+
|                 row|idx|
+--------------------+---+
|[>ALPB01000001.1 ...|  0|
|[GACACTCATCCAATTT...|  1|
|[AGAAAAAAATTTACTC...|  2|
|[GAACTGATATTGCTAA...|  3|
|[GCCAGATATGGAGAAG...|  4|
|[CATACCTATTATCGAG...|  5|
|[CAAATTTTATTTTGTC...|  6|
|[GCCGAACTAGATCCAA...|  7|
|[AGGAAAAATTGATAGA...|  8|
|[TGGGTTTTGAAATTAA...|  9|
|[TGGGTTGGTCCAACAC...| 10|
|[TGATCCTGTTGGAGAA...| 11|
|[TGAATCTGAAAGCCCT...| 12|
|[CGAAAATGCCATGTTA...| 13|
|[TATAGGTAAAATCGGA...| 14|
|[AAGCAGAAATAGTTGT...| 15|
|[GAAGTTAAATTTATTG...| 16|
|[>ALPB01000002.1 ...| 17|
|[CATTTCTTTAGGTATT...| 18|
|[AACTCAATCAATTTGA...| 19|
+--------------------+---+
only showing top 20 rows



### Parse dos arquivos FASTA

os arquivos [FASTA]([FASTA](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp)), tem o seguinte formato:

```
>ID.CONTIG
ATTC....
GCG...
CCG...
>ID2.CONTIG
GGC...
...
```

nesta primeira sessão fazermos um parse desses arquivos para agrupar as sequẽncias por ID, calcular os kmers para esses contigs e obter um map com as freqências dos kmers em todos os contigs de uma sequẽncia.

In [11]:
def parse_fasta_id_line(l):
    """
    Desejamos extrair os IDs das sequências da linhas que começarem pelo caracter ''>'. Pelo padrão
    FASTA, o ID é a primeira palavra e é um campo composto por ID.CONTIG
    
    Input>
        l: Uma linha de um arquivo FASTA
    Return:
        ID: da sequência ignorando o número de contigs, ou None caso não seja uma linha de ID
    """
    if l[0][0] == ">":
        heaer_splits = l[0][1:].split(" ")[0]
        seq_id_split = heaer_splits.split(".")
        return seq_id_split[0]
    else:
        return None
seq2kmer_udf = udf(parse_fasta_id_line, T.StringType())

In [12]:
fasta_null_ids_df = fasta_plain_df.withColumn("seqID_wNull", seq2kmer_udf("row"))

inspecionar o resultado

In [13]:
fasta_null_ids_df.show()

+--------------------+---+------------+
|                 row|idx| seqID_wNull|
+--------------------+---+------------+
|[>ALPB01000001.1 ...|  0|ALPB01000001|
|[GACACTCATCCAATTT...|  1|        null|
|[AGAAAAAAATTTACTC...|  2|        null|
|[GAACTGATATTGCTAA...|  3|        null|
|[GCCAGATATGGAGAAG...|  4|        null|
|[CATACCTATTATCGAG...|  5|        null|
|[CAAATTTTATTTTGTC...|  6|        null|
|[GCCGAACTAGATCCAA...|  7|        null|
|[AGGAAAAATTGATAGA...|  8|        null|
|[TGGGTTTTGAAATTAA...|  9|        null|
|[TGGGTTGGTCCAACAC...| 10|        null|
|[TGATCCTGTTGGAGAA...| 11|        null|
|[TGAATCTGAAAGCCCT...| 12|        null|
|[CGAAAATGCCATGTTA...| 13|        null|
|[TATAGGTAAAATCGGA...| 14|        null|
|[AAGCAGAAATAGTTGT...| 15|        null|
|[GAAGTTAAATTTATTG...| 16|        null|
|[>ALPB01000002.1 ...| 17|ALPB01000002|
|[CATTTCTTTAGGTATT...| 18|        null|
|[AACTCAATCAATTTGA...| 19|        null|
+--------------------+---+------------+
only showing top 20 rows



In [15]:
num_ids = fasta_null_ids_df.where(F.col("seqID_wNull").isNotNull()).count()
print("número de seuências para serem processadas", num_ids)

número de seuências para serem processadas 108


desejamos fazer um "fillna" com o último valor não nulo encontrado na coluna de sequência, para isso usaremos um operador de janela deslizante em cima do índice que serve para manter a ordem original das linhas

In [16]:
fasta_n_filter_df = fasta_null_ids_df.withColumn(
    "seqID", F.last('seqID_wNull', ignorenulls=True)\
    .over(Window\
    .orderBy('idx')\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)))

A seguir devemos excluir as linhas de header e renomear as colunas excluíndo as que não foram utilizadas

In [73]:
fasta_n_filter_df.show()

+--------------------+---+------------+------------+
|                 row|idx| seqID_wNull|       seqID|
+--------------------+---+------------+------------+
|[>ALPB01000001.1 ...|  0|ALPB01000001|ALPB01000001|
|[GACACTCATCCAATTT...|  1|        null|ALPB01000001|
|[AGAAAAAAATTTACTC...|  2|        null|ALPB01000001|
|[GAACTGATATTGCTAA...|  3|        null|ALPB01000001|
|[GCCAGATATGGAGAAG...|  4|        null|ALPB01000001|
|[CATACCTATTATCGAG...|  5|        null|ALPB01000001|
|[CAAATTTTATTTTGTC...|  6|        null|ALPB01000001|
|[GCCGAACTAGATCCAA...|  7|        null|ALPB01000001|
|[AGGAAAAATTGATAGA...|  8|        null|ALPB01000001|
|[TGGGTTTTGAAATTAA...|  9|        null|ALPB01000001|
|[TGGGTTGGTCCAACAC...| 10|        null|ALPB01000001|
|[TGATCCTGTTGGAGAA...| 11|        null|ALPB01000001|
|[TGAATCTGAAAGCCCT...| 12|        null|ALPB01000001|
|[CGAAAATGCCATGTTA...| 13|        null|ALPB01000001|
|[TATAGGTAAAATCGGA...| 14|        null|ALPB01000001|
|[AAGCAGAAATAGTTGT...| 15|        null|ALPB010

In [74]:
fasta_df = fasta_n_filter_df\
                .where(F.col("seqID_wNull").isNull())\
                .select("seqID","row")\
                .toDF("seqID","seq")

O Dataframe tratado tem o seguinte esquema

In [18]:
fasta_df.printSchema()

root
 |-- seqID: string (nullable = true)
 |-- seq: struct (nullable = true)
 |    |-- row: string (nullable = true)



inspeção do daframe

In [93]:
fasta_per_seq_df = fasta_df.rdd\
            .map(lambda r: (r.seqID, r.seq[0]))\
            .reduceByKey(lambda x,y:x+y)\
            .map(lambda x: Row(seqID=x[1],seq=x[0]))\
            .toDF(["seqID", "seq"])

In [148]:
fasta_per_seq_df.printSchema()

root
 |-- seqID: string (nullable = true)
 |-- seq: string (nullable = true)



In [94]:
fasta_per_seq_df.show()

+------------+--------------------+
|       seqID|                 seq|
+------------+--------------------+
|ALPB01000001|GACACTCATCCAATTTT...|
|ALPB01000002|CATTTCTTTAGGTATTG...|
|ALPB01000003|TTAAGTAATGTAGTACC...|
|ALPB01000004|GAACCATTAAAGGGGCT...|
|ALPB01000005|GTAAGCATTGTCATTTA...|
|ALPB01000006|GCCCAACCATTAAAACG...|
|ALPB01000007|TTCGCCTTTTAAGTAAT...|
|ALPB01000008|TATTGAAGAAGGGACTT...|
|ALPB01000009|CTTTCTCCAATTAAAAC...|
|ALPB01000010|GAGAAGTTGTAAATACA...|
|ALPB01000011|GAGCATAATATACCTCC...|
|ALPB01000012|TTTAAATTGATATTATT...|
|ALPB01000013|TTAATTTATTATTTTCA...|
|ALPB01000014|CATTAATGCTTGGCTCG...|
|ALPB01000015|ATTTTGTTTTATCCTTT...|
|ALPB01000016|CGTTGCACCTTTTGAAT...|
|ALPB01000017|CCTAAATGCACAAAAGA...|
|ALPB01000018|TTATGAAAGAAGATTTT...|
|ALPB01000019|AGTTTACACAATTATAA...|
|ALPB01000020|TTCGGATTTGTTGGATC...|
+------------+--------------------+
only showing top 20 rows



### Calculate Kmers

Nesta sessão faremos o cálculo dos [kmers](https://en.wikipedia.org/wiki/K-mer) de tambo ```K```. O objetivo é associar cada ID de sequência ao conjunto de kmers distiontos presentes em todos os seus motifs

In [95]:
K = 3

In [96]:
Seq2kmerTy = T.ArrayType(T.StringType())
def seq2kmer(seq_):
    global K
    value = seq_.strip()
    num_kmers = len(value) - K + 1
    kmers_list = [value[n*K:K*(n+1)] for n in range(0, num_kmers)]
    
    # return len(value)
    return kmers_list

seq2kmer_udf = udf(seq2kmer,Seq2kmerTy)

In [97]:
fasta_kmers_df = fasta_per_seq_df\
        .withColumn("kmers", seq2kmer_udf("seq"))\

In [98]:
fasta_kmers_df.printSchema()

root
 |-- seqID: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- kmers: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [99]:
fasta_kmers_df.show()

+------------+--------------------+--------------------+
|       seqID|                 seq|               kmers|
+------------+--------------------+--------------------+
|ALPB01000001|GACACTCATCCAATTTT...|[GAC, ACT, CAT, C...|
|ALPB01000002|CATTTCTTTAGGTATTG...|[CAT, TTC, TTT, A...|
|ALPB01000003|TTAAGTAATGTAGTACC...|[TTA, AGT, AAT, G...|
|ALPB01000004|GAACCATTAAAGGGGCT...|[GAA, CCA, TTA, A...|
|ALPB01000005|GTAAGCATTGTCATTTA...|[GTA, AGC, ATT, G...|
|ALPB01000006|GCCCAACCATTAAAACG...|[GCC, CAA, CCA, T...|
|ALPB01000007|TTCGCCTTTTAAGTAAT...|[TTC, GCC, TTT, T...|
|ALPB01000008|TATTGAAGAAGGGACTT...|[TAT, TGA, AGA, A...|
|ALPB01000009|CTTTCTCCAATTAAAAC...|[CTT, TCT, CCA, A...|
|ALPB01000010|GAGAAGTTGTAAATACA...|[GAG, AAG, TTG, T...|
|ALPB01000011|GAGCATAATATACCTCC...|[GAG, CAT, AAT, A...|
|ALPB01000012|TTTAAATTGATATTATT...|[TTT, AAA, TTG, A...|
|ALPB01000013|TTAATTTATTATTTTCA...|[TTA, ATT, TAT, T...|
|ALPB01000014|CATTAATGCTTGGCTCG...|[CAT, TAA, TGC, T...|
|ALPB01000015|ATTTTGTTTTATCCTTT

inspeção do daframe

In [100]:
fasta_kmers_df.printSchema()

root
 |-- seqID: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- kmers: array (nullable = true)
 |    |-- element: string (containsNull = true)



Para validação, podemos obter estatísticas básicas dso kmers obtidos. Para isso vamos contar o número de kmers por ID de sequência e obter um describe da coluna

In [101]:
n_kmers_df = fasta_kmers_df\
                    .withColumn("n_kmers", size(col("kmers")))\
                    .select("n_kmers")\

In [118]:
kmers_pofile_df = fasta_kmers_df.select("seqID","kmers")

### Extração de features

O número de K que defie o tamanho dos k-mers define um espaço de features de dimensão $4^K$, para codificar essas features podemos usar a classe ```CountVectorizer```. Essa codificação atribui ordinais a cada kmer único e cria duas listas para representar a presença e o frequência absoluta dos mesmos

In [123]:
from pyspark.ml.feature import CountVectorizer

In [124]:
kmers_pofile_df.printSchema()

root
 |-- seqID: string (nullable = true)
 |-- kmers: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [139]:
%%time
cv = CountVectorizer(inputCol="kmers", outputCol="features")

model = cv.fit(kmers_pofile_df)

features_df = model.transform(kmers_pofile_df)

CPU times: user 6.58 ms, sys: 3.25 ms, total: 9.83 ms
Wall time: 1.04 s


In [140]:
## conferir resultado temporário
features_df.select("seqID","features").toPandas().to_csv('features.csv')

In [142]:
%%time
unique_features_count = features_df.select("features").distinct().count()
print("Número de features únicas ",unique_features_count )

Número de features únicas  108
CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 1.07 s


In [143]:
print("%d das %d sequências tem features únicas" % (unique_features_count, num_ids))

108 das 108 sequências tem features únicas


## Clustering

Para o ajuste dos hiperparâmetros da clusterização devemos fazer um parameter sweep para achar o número ideal de clusters. A avaliação da qualidade do cluster é dada pela [Métreica de Silhouette](https://spark.apache.org/docs/2.3.1/api/java/org/apache/spark/ml/evaluation/ClusteringEvaluator.html)

In [144]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [145]:
bkm = BisectingKMeans()
# model = bkm.fit(features_df)
clustering_pipeline = Pipeline(stages=[bkm])

In [149]:
%%time
paramGrid = ParamGridBuilder() \
    .addGrid(bkm.k, [5, 10, 20, 50, 70, 100]) \
    .build()

crossval = CrossValidator(estimator=clustering_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=ClusteringEvaluator(),
                          numFolds=5)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel= crossval.fit(features_df)

CPU times: user 1.26 s, sys: 527 ms, total: 1.79 s
Wall time: 4min 11s


In [150]:
cluster_df = cvModel.transform(features_df)

In [151]:
cluster_df.show()

+------------+--------------------+--------------------+----------+
|       seqID|               kmers|            features|prediction|
+------------+--------------------+--------------------+----------+
|ALPB01000001|[GAC, ACT, CAT, C...|(83,[0,1,2,3,4,6,...|        25|
|ALPB01000002|[CAT, TTC, TTT, A...|(83,[0,1,2,3,4,5,...|        81|
|ALPB01000003|[TTA, AGT, AAT, G...|(83,[0,1,2,3,4,5,...|        45|
|ALPB01000004|[GAA, CCA, TTA, A...|(83,[0,1,2,3,4,5,...|        72|
|ALPB01000005|[GTA, AGC, ATT, G...|(83,[0,1,2,4,5,6,...|         4|
|ALPB01000006|[GCC, CAA, CCA, T...|(83,[0,1,2,3,4,5,...|        56|
|ALPB01000007|[TTC, GCC, TTT, T...|(83,[0,1,2,3,4,5,...|        77|
|ALPB01000008|[TAT, TGA, AGA, A...|(83,[0,1,2,3,4,5,...|        53|
|ALPB01000009|[CTT, TCT, CCA, A...|(83,[0,1,2,3,4,5,...|        55|
|ALPB01000010|[GAG, AAG, TTG, T...|(83,[0,1,2,3,4,5,...|        83|
|ALPB01000011|[GAG, CAT, AAT, A...|(83,[0,1,2,3,4,5,...|         1|
|ALPB01000012|[TTT, AAA, TTG, A...|(83,[0,1,2,3,

In [153]:
cluster_df.select("prediction").describe().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|               108|
|   mean|47.398148148148145|
| stddev|29.315218788427305|
|    min|                 0|
|    max|                99|
+-------+------------------+

